In [1]:
from torch_geometric.data import Data
from torch_geometric.loader import ClusterData, ClusterLoader
def load_data(self):
    config = self.config
    data_ = dataset_network.get_dataset(config)
    if config.dataset == 'icdm_test':
        features, edge_index, node_index = data_.data_tensor()
        return edge_index, features, node_index
    else:
        features, true_clusters, edge_index, node_index = data_.data_tensor()
        return edge_index, features, true_clusters, node_index
cluster_data_save_dir = '/home/chensiqi/lhtz/ACMin/data/icdm2023_session1_test/processed'
edge_index, features, node_index = self.load_data()
network_data = Data(x=features, edge_index=edge_index, node_index = node_index)
cluster_data = ClusterData(network_data, num_parts=10, recursive=True, save_dir=cluster_data_save_dir)
data_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True, num_workers=16)
for i, batch in enumerate(data_loader):
    print(batch.x.shape[0])


NameError: name 'self' is not defined

In [4]:
from sklearn.cluster import KMeans
import numpy as np
import os
# import cPickle as pickle
import pickle
import importlib
import torch
# from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
# from scipy.sparse.linalg.eigen.arpack import eigs as largest_eigs
from scipy.linalg import qr
import time
from scipy.sparse import csc_matrix, csr_matrix
from numpy import linalg as LA
import random
from torch_geometric.data import Data
from torch_geometric.loader import ClusterData, ClusterLoader
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
from tqdm import tqdm
import dataset_network

from config.networks import default as cfg

def solve_envs():
    mod = importlib.import_module(f'config.networks')
    cfg = getattr(mod, 'default')
    print(cfg)
    # update config
    for k, v in cfg.items():
            print(k, '\t', v)
    return cfg

def load_data(cfg):
    config = cfg
    data_ = dataset_network.get_dataset(config)
    if config.dataset == 'icdm_test':
        features, edge_index, node_index = data_.data_tensor()
        return edge_index, features, node_index
    else:
        features, true_clusters, edge_index, node_index = data_.data_tensor()
        return edge_index, features, true_clusters, node_index

if __name__ == '__main__':
    cfg = solve_envs()
    edge_index, features, node_index = load_data(cfg)
    network_data = Data(x=features, edge_index=edge_index, node_index = node_index)
    cluster_data_save_dir = 'cluster_data/'
    cluster_data = ClusterData(network_data, num_parts=10, recursive=False, save_dir=cluster_data_save_dir)
    data_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True, num_workers=16)
    del cluster_data
    del network_data
    num_batchs = len(data_loader)
    available_list = [i for i in range(features.shape[0])]
    print(f'origin_len: {len(available_list)}')
    node_dict = dict()
    for i, batch in enumerate(data_loader):
        def get_index(ith):
            nonzero_indices = torch.nonzero(torch.all(batch.x[ith] == features, dim=1))
            if nonzero_indices.shape[0]>1:
                for i_repeat in range(nonzero_indices.shape[0]):
                    ind_repeat = nonzero_indices[i_repeat].item()
                    if ind_repeat in available_list:
                        available_list.remove(ind_repeat)
                        return ind_repeat
            else:
                print(nonzero_indices.item())
                print(batch.node_index[ith])
                return nonzero_indices.item()       
        fs_list = [inde for inde in range(batch.x.shape[0])]
        fs_list = fs_list[:10]
        for index_fs in fs_list:
            get_index(index_fs)
    print(f'after_len: {len(available_list)}')
    file_path = "/home/chensiqi/lhtz/ACMin/data/icdm2023_session1_test/processed/node_list.pkl"
    with open(file_path, "wb") as file_:
        pickle.dump(node_dict, file_)


ACMin_cluster_params 	 {'alpha': 0.2, 'beta': 0.35, 't': 5, 'tmax': 200, 'ri': False, 'print_batch': 20}
batch_size 	 1
cluster_parts_num 	 10
cluster_result_path 	 results
data_total_path 	 data
dataset 	 icdm_test
mode 	 cluster
name 	 default
num_cluster 	 200


/home/chensiqi/lhtz/ACMin/dataset_network/base_network.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.x, dtype = torch.float), torch.tensor(edges_df), torch.tensor(self.nodes)


origin_len: 2365188
648905
tensor(648905)
444396
tensor(444396)
419476
tensor(419476)
1856230
tensor(1856230)
1854316
tensor(1854316)
123096
tensor(123096)
444337
tensor(444337)
617412
tensor(617412)
2242204
tensor(2242204)
1854520
tensor(1854520)
2308658
tensor(2308658)
1115632
tensor(1115632)
1457024
tensor(1457024)
1580535
tensor(1580535)
1119911
tensor(1119911)
1818160
tensor(1818160)
1439799
tensor(1439799)
1098138
tensor(1098138)
118509
tensor(118509)
1712268
tensor(1712268)
1747867
tensor(1747867)
1688068
tensor(1688068)
2360632
tensor(2360632)
1632194
tensor(1632194)
1665956
tensor(1665956)
1297036
tensor(1297036)
1675841
tensor(1675841)
1665826
tensor(1665826)
2279270
tensor(2279270)
2220812
tensor(2220812)
1100
tensor(1100)
1060
tensor(1060)
154573
tensor(154573)
2221153
tensor(2221153)
174314
tensor(174314)
232570
tensor(232570)
181355
tensor(181355)
2219989
tensor(2219989)
2236846
tensor(2236846)
842150
tensor(842150)
1357
tensor(1357)
1688576
tensor(1688576)
1649159
tensor

In [3]:
import pandas as pd
file_path = '/home/chensiqi/lhtz/ACMin/data/icdm2023_session1_test/icdm2023_session1_test_node_feat.txt'
feat_df = pd.read_csv(file_path, sep=',', header=None)
feat_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.207364,0.200824,-0.206365,0.251794,-0.069225,0.142702,0.333365,0.064161,0.258544,-0.223119,...,-0.172212,0.293582,-0.506954,-0.139091,-0.137190,-0.462928,-0.127135,-0.000702,-0.242490,0.076512
1,0.511876,0.969503,-0.027224,-0.198855,0.436524,-0.016676,-0.466111,0.751885,-0.468856,-0.566030,...,0.170765,0.440941,-1.620551,0.841428,-1.306823,0.206225,0.189508,-1.100875,0.470120,0.146060
2,-0.730822,0.303364,-0.031425,0.201133,0.418901,0.532799,-0.289830,0.420402,-0.455646,0.682239,...,0.037289,0.521415,-0.151157,0.248995,-0.076819,-0.896406,0.172010,0.928897,0.654002,0.329679
3,0.445594,0.141923,0.281454,0.474735,0.074478,0.212980,0.450492,0.125644,0.281054,0.196386,...,-0.099492,0.064200,-0.396112,0.474283,0.151561,-0.587608,-0.063882,-0.092721,0.461798,0.063294
4,0.192593,-0.076425,-0.130310,0.169446,0.171604,0.284075,0.017297,-0.178282,0.245168,-0.121027,...,0.136974,0.412531,-0.120065,-0.362007,-0.036251,0.165670,0.128652,0.048835,0.132938,-0.006432
5,-0.008065,-0.138921,-0.064647,0.252782,0.001114,-0.244563,0.167826,-0.223832,0.202851,-0.181798,...,0.099385,-0.106262,0.081262,0.511171,0.247757,-0.228312,-0.018343,-0.380453,0.293014,-0.039101
6,0.027900,0.137569,0.246875,0.085310,0.312850,-0.367335,0.399071,-0.454076,-0.253019,0.508809,...,-0.227704,0.358255,0.604306,0.151067,-0.470931,-1.394685,0.644307,1.856121,1.849012,-0.253681
7,-0.055887,-0.519914,-0.062301,0.101411,0.174484,-0.437628,0.170330,-0.822256,0.197434,-0.320361,...,0.165907,0.378982,-0.411608,0.524655,-0.487708,0.531333,-0.431509,-0.447767,-0.659172,0.224975
8,-0.117591,-0.011554,-0.040016,0.014770,-0.049148,0.160069,0.081594,-0.045521,0.089562,0.085754,...,0.018401,-0.026867,0.088138,0.062426,0.095858,-0.037927,-0.014244,0.001102,0.020652,-0.100472
9,-0.321493,-0.254758,-0.399477,-0.108316,-0.302608,-0.057603,-0.455503,0.976187,1.427465,-0.240252,...,0.430297,1.427086,-0.742448,-0.201521,-1.337251,-0.035177,0.671922,-2.049084,-0.174701,1.233188
